# Text Similarity
(by Tevfik Aytekin)

In [140]:
from nltk.corpus import wordnet as wn
import nltk
from nltk.corpus import gutenberg, brown
import numpy as np
from scipy.sparse import csr_matrix
from nltk.tokenize import RegexpTokenizer
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize 
from sklearn.feature_extraction.text import CountVectorizer

# WordNet

### Lexical Matrix
table taken from [https://wordnetcode.princeton.edu/5papers.pdf](https://wordnetcode.princeton.edu/5papers.pdf)

<img src="./images/lexical_matrix.png" style="width: 400px;"/>


The word meaning $M_1$ in above table can be represented by the set of word forms that can be used to express it: {F1, F2, . . . }. These sets are called synonym sets (or simply synsets).

### WordNet Hierarchy
Below is a simplified illustration of the hierarchy of wordnet.

<img src="./images/wordnet_hierarchy.png" style="width: 400px;"/>


### A word is a set of meanings
wn.synsets('word') gives these meanings.

In [141]:
wn.synsets("car")

[Synset('car.n.01'),
 Synset('car.n.02'),
 Synset('car.n.03'),
 Synset('car.n.04'),
 Synset('cable_car.n.01')]

### Synsets

A synset is a set of synonyms (word forms). Each synset corresponds to a concept/meaning. The nodes in the WordNet hierarchy corresponds to synsets. A synset is identified with a 3-part name of the form: word.pos.nn. For example, 'car.n.01' means the first meaning of 'car' used as a noun.

In [142]:
print(wn.synset('car.n.01').definition())

a motor vehicle with four wheels; usually propelled by an internal combustion engine


Lemmas correspond to word forms.

In [143]:
wn.synset('car.n.01').lemma_names()

['car', 'auto', 'automobile', 'machine', 'motorcar']

In [144]:
wn.synset('car.n.01').lemmas()

[Lemma('car.n.01.car'),
 Lemma('car.n.01.auto'),
 Lemma('car.n.01.automobile'),
 Lemma('car.n.01.machine'),
 Lemma('car.n.01.motorcar')]

In [145]:
wn.synset('car.n.02').lemma_names()

['car', 'railcar', 'railway_car', 'railroad_car']

Hypernyms anf hyponyms

In linguistics, hyponymy means a subtype and a hypernym means a supertype

In [146]:
wn.synset('car.n.01').hypernyms()

[Synset('motor_vehicle.n.01')]

In [147]:
wn.synset('car.n.01').hyponyms()

[Synset('ambulance.n.01'),
 Synset('beach_wagon.n.01'),
 Synset('bus.n.04'),
 Synset('cab.n.03'),
 Synset('compact.n.03'),
 Synset('convertible.n.01'),
 Synset('coupe.n.01'),
 Synset('cruiser.n.01'),
 Synset('electric.n.01'),
 Synset('gas_guzzler.n.01'),
 Synset('hardtop.n.01'),
 Synset('hatchback.n.01'),
 Synset('horseless_carriage.n.01'),
 Synset('hot_rod.n.01'),
 Synset('jeep.n.01'),
 Synset('limousine.n.01'),
 Synset('loaner.n.02'),
 Synset('minicar.n.01'),
 Synset('minivan.n.01'),
 Synset('model_t.n.01'),
 Synset('pace_car.n.01'),
 Synset('racer.n.02'),
 Synset('roadster.n.01'),
 Synset('sedan.n.01'),
 Synset('sport_utility.n.01'),
 Synset('sports_car.n.01'),
 Synset('stanley_steamer.n.01'),
 Synset('stock_car.n.01'),
 Synset('subcompact.n.01'),
 Synset('touring_car.n.01'),
 Synset('used-car.n.01')]

In [148]:
wn.synset('car.n.01').root_hypernyms()

[Synset('entity.n.01')]

## Synonymy

### Path Similarity
path_similarity assigns a score in the range 0–1 based on the shortest path that connects the concepts in the hypernym hierarchy (-1 is returned in those cases where a path cannot be found)

In [149]:
right = wn.synset('right_whale.n.01')
orca = wn.synset('orca.n.01')
minke = wn.synset('minke_whale.n.01')
tortoise = wn.synset('tortoise.n.01')
novel = wn.synset('novel.n.01')

In [150]:
print(right.path_similarity(minke))
print(right.path_similarity(orca))
print(right.path_similarity(tortoise))
print(right.path_similarity(novel))


0.25
0.16666666666666666
0.07692307692307693
0.043478260869565216


<img src="images/wordnet_hierarchy.png" style="width: 400px;"/>

In [151]:
motorcar = wn.synset('car.n.01')
compact = wn.synset('compact.n.03')
hatchback = wn.synset('hatchback.n.01')
print(motorcar.path_similarity(compact))

0.5


In [152]:
print(hatchback.path_similarity(compact))

0.3333333333333333


In [153]:
print(hatchback.path_similarity(hatchback))

1.0


## Automated ways for finding synonyms

WordNet is constructed manually by experts of linguistics. There is also the computational approach to semantics. Below we will look at one such approach for finding synonyms. The approach relies on the below fundamental hypothesis:
<br><br>
<center><b>Distributional Hypothesis: similar words appear in similar contexts.</b></center>
<br><br>
We will first need to build a corpus and a co-occurrence matrix.

In [154]:
def build_corpus(text):
    """ 
  
    Parameters: 
    text (string): A (long) string 
  
    Returns: 
    words: A list of unique word names.
    word_to_index: a mapping from word names to integers.
    index_to_word: a mapping from integers to word names.
  
    """
    porter = nltk.PorterStemmer()

    tokenizer = RegexpTokenizer(r'\w+')
    words = [] 
    for i in sent_tokenize(text): 
        for j in tokenizer.tokenize(i):
            words.append(j.lower())
    words = np.unique(words)

    porter = nltk.PorterStemmer()
    words = [porter.stem(t) for t in words]
    #wn_lemma = nltk.WordNetLemmatizer()
    #words = [wn_lemma.lemmatize(t) for t in words]

    words = np.unique(words)
    
    word_to_index = {}
    index_to_word = {}
    counter = 0;
    for w in words:
        word_to_index[w] = counter
        index_to_word[counter] = w
        counter += 1  
    return words, word_to_index, index_to_word

In [155]:
text = "This is data mining course cmp5101. It is about data mining. I like it so much."
words, word_to_index, index_to_word = build_corpus(text)

In [156]:
corpus_size = len(words)
print(words)
print(word_to_index)
print(index_to_word)

['about' 'cmp5101' 'cours' 'data' 'i' 'is' 'it' 'like' 'mine' 'much' 'so'
 'thi']
{'about': 0, 'cmp5101': 1, 'cours': 2, 'data': 3, 'i': 4, 'is': 5, 'it': 6, 'like': 7, 'mine': 8, 'much': 9, 'so': 10, 'thi': 11}
{0: 'about', 1: 'cmp5101', 2: 'cours', 3: 'data', 4: 'i', 5: 'is', 6: 'it', 7: 'like', 8: 'mine', 9: 'much', 10: 'so', 11: 'thi'}


In [157]:
word_to_index['data']

3

In [158]:
index_to_word[3]

'data'

In [159]:
text = brown.raw()
words, word_to_index, index_to_word = build_corpus(text)
corpus_size = len(words)
print(corpus_size)
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(text)
print("number of tokens: ", len(tokens))
words[1500:1510]

27114
number of tokens:  2084675


array(['alcov', 'alden', 'alderman', 'aldermen', 'aldo', 'aldridg', 'ale',
       'alec', 'aleck', 'alemagna'], dtype='<U20')

In [115]:
def build_co_matrix2(text, words, word_to_index, window=1):
    """ 
    Build a co-occurrence matrix 
    
    Parameters: 
    text (string): A long string to be split into sentences.
    words: A list of unique word names.
    word_to_index: a mapping from word names to integers.
    window: The size of the context window.
  
    Returns: 
    co_matrix: ndarray 
  
    """
    porter = nltk.PorterStemmer()
    wn_lemma = nltk.WordNetLemmatizer()

    tokenizer = RegexpTokenizer(r'\w+')
    corpus_size = len(words)
    co_matrix = np.zeros((corpus_size,corpus_size),dtype=int)
    for s in sent_tokenize(text): 
        sent = [] 
        for w in tokenizer.tokenize(s):        
            sent.append(porter.stem(w.lower()))
            #sent.append(wn_lemma.lemmatize(w.lower()))
        for i, w in enumerate(sent):
            for j in range(max(i-window,0),min(i+window+1,len(sent))):
                co_matrix[word_to_index[w],word_to_index[sent[j]]] += 1
        np.fill_diagonal(co_matrix,0)
    return co_matrix

The following code is not an efficient way to build a co-occurrence matrix, it will be used for illustration

In [116]:
def build_co_matrix(text, window=1):
    porter = nltk.PorterStemmer()
    wn_lemma = nltk.WordNetLemmatizer()

    tokenizer = RegexpTokenizer(r'\w+')
    counter = 0
    co_matrix = pd.DataFrame();
    for s in sent_tokenize(text): 
        sent = [] 
        for w in tokenizer.tokenize(s):        
            sent.append(porter.stem(w.lower()))
            #sent.append(wn_lemma.lemmatize(w.lower()))
        for i, w in enumerate(sent):
            for j in range(max(i-window,0),min(i+window+1,len(sent))):
                if w == sent[j]:# skip the word itself
                    co_matrix.loc[w,sent[j]] = 0
                elif (w in co_matrix.index and sent[j] in co_matrix.columns) and not np.isnan(co_matrix.loc[w,sent[j]]):
                    co_matrix.loc[w,sent[j]] += 1
                else:
                    co_matrix.loc[w,sent[j]] = 1
    co_matrix.fillna(0, inplace=True)
    return co_matrix

How tokenization with regex works

In [117]:
text = "This is data mining course cmp5101. It is about data mining. I like it so much."
tokenizer = RegexpTokenizer(r'\w+')
for w in tokenizer.tokenize(text):  
    print(w)

This
is
data
mining
course
cmp5101
It
is
about
data
mining
I
like
it
so
much


In [118]:
matrix = build_co_matrix(text, 2)
matrix

,thi,is,data,mine,cours,cmp5101,it,about,i,like,so,much
thi,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
is,1.0,0.0,2.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
data,1.0,2.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
mine,0.0,1.0,2.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
cours,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
cmp5101,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
it,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
about,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
i,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
like,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [119]:
words, word_to_index, index_to_word = build_corpus(text)
matrix = build_co_matrix2(text, words, word_to_index, 2)
matrix

array([[0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 1, 0, 0, 2, 0, 0, 2, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [1, 0, 0, 2, 0, 0, 1, 0, 1, 0, 0, 1],
       [1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0],
       [1, 1, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]])

Now let us build the co-occurrence matrix for the entire text.

In [120]:
text = brown.raw()
words, word_to_index, index_to_word = build_corpus(text)
co_matrix = build_co_matrix2(text, words, word_to_index, 5)
print(co_matrix.shape)
print(co_matrix)

(27114, 27114)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


### Cosine Similarity:
Intuition: Dot product increases if both pairs have the same sign and decreases if pairs have different signs (similar to correlation, actually Pearson correlation is just cosine similarity of the mean centered vectors). Division by the norms is necessary to penalize vectors which has large values.

In [121]:
co_matrix[:,0][np.nonzero(co_matrix[:,0])]

array([  2,   1,   2,   1,   1,   2,   1,   1,   1,   2,  36,   2,   1,
         1,   2,   1,   1,   3,   1,   4,   1,   1,   1,   1,   2,  17,
         1,   3,   2,   2,   8,   1,   2,   1,  11,   1,   1,  17,   2,
         1,   5,   1,   1,   2,   1,   1,   1,   7,   4,   2,   1,   1,
        19,   1,   9,   1,   1,   1,   1,  23,   1,   1,   1,   1,   1,
         1,   6,   1,  26,   1,   1,   5,   1,   1,   7,   2,   1,  73,
         2,   1,   1,   2,  13,   1,   1,   1,   4,   1,   6,   1,  16,
         3,   1,   1,   2,   1,   1,   3,   3,   2,   4,   1,   3,   1,
         1,   1,   1,  45, 172,   1,   1,   2,   2,   1,   2,   1,   1,
         1,   1,   1,   1,   1,   3,   1,   1,   1,   1,   3,   1,   3,
         1,   2,   1,   1,   5,   1,   1,   1,   1,   1,   1,   5,   1,
         1,   1,   1,   1,   1,   1,   1,   2,   1,   1,   5,   2,   1,
         1,   1,   2,   1,   1,   1,   1,   1,   1,   1,   1,   3,   1,
         1,   2,   3,   9,   1,   1,   1,   2,   3,   3,   9,   

In [ ]:
# Finds cosine similarity between two vectors a and b
def cosine(a, b):
    dot = np.dot(a, b)
    norma = np.linalg.norm(a)
    normb = np.linalg.norm(b)
    return dot / (norma * normb)

Find most similar words to the target word using cosine similarity

In [122]:
target = word_to_index['book']
target

3621

In [123]:
word_vector = co_matrix[target,:]
word_vector.shape

(27114,)

In [124]:
word_vector = np.reshape(word_vector,(word_vector.size,1 ))
word_vector.shape

(27114, 1)

In [125]:
sims = np.dot(word_vector.T,co_matrix)
sims = sims[0,:]
sims

array([ 370406,   92920, 1378577, ...,    8263,    2963,    6049])

In [126]:
sims.argsort()[::-1][:10]

array([12229, 16505,  2343, 23958, 13061, 16875,  4753, 24296, 16706,
       24294])

In [127]:
index_to_word[12229]

'in'

In [128]:
def incremental_row_norms(matrix):
    row_norms = []
    for row in matrix:
        row_norm = np.sqrt(np.dot(row, row))
        row_norms.append(row_norm)
    return row_norms

In [129]:
norms = incremental_row_norms(co_matrix.T)
norms

[423.86200584624237,
 103.88455130576442,
 1443.9882270988223,
 9.539392014169456,
 6.782329983125268,
 6.164414002968976,
 22.891046284519195,
 4.795831523312719,
 5.477225575051661,
 20.46948949045872,
 6.928203230275509,
 10.0,
 26.720778431774775,
 5.0,
 7.0710678118654755,
 16.15549442140351,
 3.3166247903554,
 5.477225575051661,
 5.477225575051661,
 34.19064199455752,
 3.7416573867739413,
 18.520259177452136,
 3.872983346207417,
 4.123105625617661,
 3.872983346207417,
 5.656854249492381,
 30.59411708155671,
 6.0,
 9.273618495495704,
 10.198039027185569,
 8.831760866327848,
 3.605551275463989,
 6.324555320336759,
 10.677078252031311,
 11.313708498984761,
 4.58257569495584,
 37.68288736283355,
 8.0,
 2988.8245181007196,
 854.3652614660781,
 428.6583721333342,
 48.80573736764972,
 7.211102550927978,
 13.379088160259652,
 6.164414002968976,
 8.06225774829855,
 10.677078252031311,
 6.164414002968976,
 5.830951894845301,
 64.66838485689897,
 34.49637662132068,
 18.867962264113206,
 5.8

In [130]:
norm_sims = np.divide(sims,norms)

In [131]:
norm_sims.argsort()[::-1][:10]

array([ 3621, 15247,  5609, 11816, 22614, 16887, 22582, 20014,  5222,
        2200])

In [132]:
index_to_word[3621]

'book'

In [133]:
index_to_word[8957]

'famili'

In [134]:
word_vector = co_matrix[word_to_index['novel'],:]
word_vector = np.reshape(word_vector,(1,word_vector.size))
sims = np.dot(word_vector,co_matrix)
sims = sims[0,:]
norm_sims = np.divide(sims,norms)
top10 = norm_sims.argsort()[::-1][:10]
top10

array([16688, 17624,  3621, 26796,  5576,  4800, 25055, 22516, 22193,
       11590])

In [135]:
for i in range(len(top10)):
    print(index_to_word[top10[i]])

novel
part
book
world
column
center
under
spirit
societi
histori


In [ ]:
word_vector = co_matrix[word_to_index['novel'],:]
word_vector = np.reshape(word_vector,(1,word_vector.size))
sims = np.dot(word_vector,co_matrix)
sims = sims[0,:]
#norm_sims = np.divide(sims,norms)
top10 = sims.argsort()[::-1][:10]
top10

In [ ]:
for i in range(len(top10)):
    print(index_to_word[top10[i]])

In [136]:
word_vector = co_matrix[word_to_index['friend'],:]
word_vector = np.reshape(word_vector,(1,word_vector.size))
sims = np.dot(word_vector,co_matrix)
sims = sims[0,:]
norm_sims = np.divide(sims,norms)
top10 = norm_sims.argsort()[::-1][:10]
top10

array([ 9818,  9022, 15911, 26564, 17579, 22291,  7899,  9110, 11236,
       15530])

In [137]:
for i in range(len(top10)):
    print(index_to_word[top10[i]])

friend
father
mother
wife
parent
son
duti
fellow
head
mine


In [138]:
word_vector = co_matrix[word_to_index['eight'],:]
word_vector = np.reshape(word_vector,(1,word_vector.size))
sims = np.dot(word_vector,co_matrix)
sims = sims[0,:]
norm_sims = np.divide(sims,norms)
top10 = norm_sims.argsort()[::-1][:10]
top10

array([ 8091,  9689, 21894, 24112,  9327, 21405, 24858, 23834, 24840,
       15504])

In [139]:
for i in range(len(top10)):
    print(index_to_word[top10[i]])

eight
four
six
three
five
seven
two
ten
twenti
million
